In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import optuna
import warnings
warnings.filterwarnings('ignore')

optuna.logging.set_verbosity(optuna.logging.INFO) 
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

/Users/dariapavlova/Documents/defi_trends_analysis/.ven311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class DeFiVolumePredictor:
    """A comprehensive class for predicting volumes with semantic features and Optuna optimization."""
    
    def __init__(self, use_semantic_features=True):
        self.use_semantic = use_semantic_features
        self.scalers = {}
        self.models = {}
        self.results = {}
        self.best_params = {}
        self.data_for_opt = {} # To store data for Optuna

    # --- Feature Engineering (No Changes) ---
    def create_semantic_features(self, df):
        """Create semantic features based on an ontology."""
        df = df.copy()
        df['volume_ratio'] = df['altcoin_volume_24h'] / (df['total_volume_24h'] + 1e-8)
        df['btc_share'] = df['btc_dominance'] / 100
        df['LCR'] = df['volume_ratio']**2 * np.exp(-df['active_exchanges']/100)
        df['vol_to_cap'] = df['total_volume_24h'] / (df['total_market_cap'] + 1e-8)
        df['pair_token_ratio'] = df['active_market_pairs'] / (df['active_cryptocurrencies'] + 1)
        df['MDP'] = df['vol_to_cap'] * np.sqrt(df['pair_token_ratio'])
        df['vol_cv'] = df['vol_std_12'] / (df['vol_mean_12'] + 1e-8)
        df['sentiment_deviation'] = np.abs(df['value'] - 50) / 50
        df['SVA'] = df['sentiment_deviation'] * df['vol_cv']
        df['PCI'] = np.log(df['active_cryptocurrencies'] + 1) * df['pair_token_ratio']
        df['DAR'] = df['volume_ratio'] * (1 - df['btc_share'])
        df['relative_vol'] = df['alt_std_12'] / (df['vol_std_12'] + 1e-8)
        df['ILFI'] = df['relative_vol'] * df['volume_ratio']
        df['CME'] = np.log(df['active_exchanges'] + 1) * (1 - df['volume_ratio']**2)
        df['alt_to_cap'] = df['altcoin_volume_24h'] / (df['altcoin_market_cap'] + 1e-8)
        df['LV'] = (df['vol_to_cap'] + df['alt_to_cap']) / 2
        semantic_cols = ['LCR', 'MDP', 'SVA', 'PCI', 'DAR', 'ILFI', 'CME', 'LV']
        for col in semantic_cols:
            df[col] = df[col].replace([np.inf, -np.inf], np.nan)
            df[col] = df[col].fillna(df[col].median())
        return df, semantic_cols
    
    def create_lag_features(self, df, target_col='log_total_volume_24h'):
        """Create correct lag features WITHOUT data leakage."""
        df = df.copy()
        lag_cols = []
        for lag in [1, 3, 6, 12, 24, 48]:
            col_name = f'{target_col}_lag_{lag}'
            df[col_name] = df[target_col].shift(lag)
            lag_cols.append(col_name)
        for window in [6, 12, 24, 48]:
            ma_col = f'{target_col}_ma_{window}'
            std_col = f'{target_col}_std_{window}'
            df[ma_col] = df[target_col].rolling(window=window, min_periods=1).mean()
            df[std_col] = df[target_col].rolling(window=window, min_periods=1).std()
            lag_cols.extend([ma_col, std_col])
        return df, lag_cols

    def prepare_data(self, df, horizon=48, validation_split_ratio=0.15):
        """Prepare data and create a validation set for Optuna."""
        df, semantic_cols = self.create_semantic_features(df)
        df, lag_cols = self.create_lag_features(df)
        base_features = [
            'log_total_cap', 'log_altcoin_cap', 'btc_dominance', 'active_exchanges', 
            'log_market_pairs', 'zscore_12', 'vol_mean_12', 'vol_std_12', 
            'vol_mean_24', 'vol_std_24'
        ]
        features = base_features + lag_cols + (semantic_cols if self.use_semantic else [])
        
        for h in range(1, horizon + 1):
            df[f'target_{h}'] = df['log_total_volume_24h'].shift(-h)
            
        df_clean = df.dropna()
        X = df_clean[features].values
        y = df_clean[[f'target_{h}' for h in range(1, horizon + 1)]].values
        
        split_idx = int(0.8 * len(X))
        X_train_full, X_test = X[:split_idx], X[split_idx:]
        y_train_full, y_test = y[:split_idx], y[split_idx:]
        
        self.scalers['X'] = RobustScaler()
        self.scalers['y'] = StandardScaler()
        
        X_train_full_scaled = self.scalers['X'].fit_transform(X_train_full)
        X_test_scaled = self.scalers['X'].transform(X_test)
        y_train_full_scaled = self.scalers['y'].fit_transform(y_train_full)
        y_test_scaled = self.scalers['y'].transform(y_test)
        
        val_split_idx = int(len(X_train_full_scaled) * (1 - validation_split_ratio))
        self.data_for_opt = {
            'X_train': X_train_full_scaled[:val_split_idx],
            'y_train': y_train_full_scaled[:val_split_idx],
            'X_val': X_train_full_scaled[val_split_idx:],
            'y_val': y_train_full_scaled[val_split_idx:]
        }
        
        return X_train_full_scaled, X_test_scaled, y_train_full_scaled, y_test_scaled, features

    # --- Optuna Objective Functions ---
    def _objective_rf(self, trial):
        """Objective function for Random Forest optimization."""
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 400),
            'max_depth': trial.suggest_int('max_depth', 5, 30),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 30),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
            'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        }
        rf = RandomForestRegressor(random_state=42, n_jobs=-1, **params)
        from sklearn.multioutput import MultiOutputRegressor
        model = MultiOutputRegressor(rf)
        
        model.fit(self.data_for_opt['X_train'], self.data_for_opt['y_train'])
        y_pred = model.predict(self.data_for_opt['X_val'])
        
        mae = mean_absolute_error(self.data_for_opt['y_val'], y_pred)
        return mae

    def _objective_rnn(self, trial, model_type='lstm'):
        """Objective function for LSTM/GRU optimization."""
        seq_len = trial.suggest_categorical('sequence_length', [24, 48, 72])
        
        X_train_seq, y_train_seq = self._create_sequences(self.data_for_opt['X_train'], self.data_for_opt['y_train'], seq_len)
        X_val_seq, y_val_seq = self._create_sequences(self.data_for_opt['X_val'], self.data_for_opt['y_val'], seq_len)
        
        if len(X_train_seq) == 0 or len(X_val_seq) == 0:
             raise optuna.exceptions.TrialPruned()

        params = {
            'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True),
            'dropout_rate': trial.suggest_float('dropout_rate', 0.1, 0.5),
            'n_units_1': trial.suggest_categorical('n_units_1', [64, 128, 256]),
            'n_units_2': trial.suggest_categorical('n_units_2', [32, 64, 128]),
        }
        
        RNN_LAYER = LSTM if model_type == 'lstm' else GRU
        
        model = Sequential([
            RNN_LAYER(params['n_units_1'], return_sequences=True, input_shape=(seq_len, X_train_seq.shape[2])),
            BatchNormalization(),
            Dropout(params['dropout_rate']),
            RNN_LAYER(params['n_units_2']),
            BatchNormalization(),
            Dense(64, activation='relu'),
            Dropout(params['dropout_rate']),
            Dense(y_train_seq.shape[1])
        ])
        
        model.compile(optimizer=Adam(learning_rate=params['learning_rate']), loss='huber', metrics=['mae'])
        
        history = model.fit(
            X_train_seq, y_train_seq,
            validation_data=(X_val_seq, y_val_seq),
            epochs=50,
            batch_size=32,
            callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
            verbose=0
        )
        
        val_mae = min(history.history['val_mae'])
        return val_mae
    
    # --- Main Optimization Runners ---
    def optimize_model(self, model_name, n_trials=30):
        """Runs Optuna study for a given model with debugging."""
        print(f"\nOptimizing {model_name.upper()}...")
        
        def print_callback(study, trial):
            print(f"Trial {trial.number} for {model_name.upper()} finished with value: {trial.value:.5f}")
            
        study = optuna.create_study(direction='minimize')
        
        if model_name == 'rf':
            study.optimize(self._objective_rf, n_trials=n_trials, callbacks=[print_callback])
        elif model_name in ['lstm', 'gru']:
            objective_func = lambda trial: self._objective_rnn(trial, model_type=model_name)
            study.optimize(objective_func, n_trials=n_trials, callbacks=[print_callback])
        else:
            raise ValueError("Unknown model name")
            
        self.best_params[model_name] = study.best_params
        print(f"\nOptimization for {model_name.upper()} complete.")
        print(f"Best validation MAE: {study.best_value:.4f}")
        print(f"Best params: {study.best_params}")
        
        self.visualize_study(study, model_name)
        
        return study.best_params

    # --- Updated Training Functions ---
    def train_model(self, model_name, X_train, y_train, X_test, y_test):
        """A unified function to train any model with optimized parameters."""
        print(f"\n=== Training {model_name.upper()} with best parameters ===")
        params = self.best_params.get(model_name, {})

        if model_name == 'rf':
            rf = RandomForestRegressor(random_state=42, n_jobs=-1, **params)
            from sklearn.multioutput import MultiOutputRegressor
            model = MultiOutputRegressor(rf)
            model.fit(X_train, y_train)
            y_pred_scaled = model.predict(X_test)
            y_test_scaled = y_test

        elif model_name in ['lstm', 'gru']:
            seq_len = params.get('sequence_length', 48)
            X_train_seq, y_train_seq = self._create_sequences(X_train, y_train, seq_len)
            X_test_seq, y_test_seq = self._create_sequences(X_test, y_test, seq_len)
            
            RNN_LAYER = LSTM if model_name == 'lstm' else GRU
            model = Sequential([
                RNN_LAYER(params.get('n_units_1', 128), return_sequences=True, input_shape=(seq_len, X_train_seq.shape[2])),
                BatchNormalization(),
                Dropout(params.get('dropout_rate', 0.2)),
                RNN_LAYER(params.get('n_units_2', 64)),
                BatchNormalization(),
                Dense(64, activation='relu'),
                Dropout(params.get('dropout_rate', 0.2)),
                Dense(y_train_seq.shape[1])
            ])
            model.compile(optimizer=Adam(learning_rate=params.get('learning_rate', 0.001)), loss='huber', metrics=['mae'])
            
            model.fit(
                X_train_seq, y_train_seq, epochs=100, batch_size=32, verbose=0,
                callbacks=[
                    EarlyStopping(patience=15, restore_best_weights=True),
                    ReduceLROnPlateau(patience=5, factor=0.5)
                ]
            )
            y_pred_scaled = model.predict(X_test_seq, verbose=0)
            y_test_scaled = y_test_seq
        else:
             raise ValueError("Unknown model name")

        y_pred = self.scalers['y'].inverse_transform(y_pred_scaled)
        y_test_orig = self.scalers['y'].inverse_transform(y_test_scaled)
        
        mae = mean_absolute_error(y_test_orig, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test_orig, y_pred))
        r2 = r2_score(y_test_orig, y_pred)
        
        print(f"Final Test Metrics -> MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}")
        self.models[model_name] = model
        self.results[model_name] = {'mae': mae, 'rmse': rmse, 'r2': r2}
        return model

    # --- Helper and Visualization ---
    def _create_sequences(self, X, y, seq_length):
        Xs, ys = [], []
        if len(X) <= seq_length: return np.array(Xs), np.array(ys)
        for i in range(seq_length, len(X)):
            Xs.append(X[i-seq_length:i])
            ys.append(y[i])
        return np.array(Xs), np.array(ys)
    
    def visualize_study(self, study, model_name):
        """Visualizes the Optuna study results."""
        try:
            fig1 = optuna.visualization.plot_optimization_history(study)
            fig1.update_layout(title_text=f'Optimization History for {model_name.upper()}')
            fig1.show()

            fig2 = optuna.visualization.plot_param_importances(study)
            fig2.update_layout(title_text=f'Hyperparameter Importance for {model_name.upper()}')
            fig2.show()
            
            fig3 = optuna.visualization.plot_slice(study)
            fig3.update_layout(title_text=f'Parameter Slices for {model_name.upper()}')
            fig3.show()

        except (ImportError, ValueError) as e:
            print(f"Could not generate plots for {model_name}: {e}. Please ensure Plotly is installed.")

In [3]:
try:
    df = pd.read_csv('df_for_learning.csv')
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values('timestamp').reset_index(drop=True)
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: 'df_for_learning.csv' not found. Please ensure it's in the correct directory.")
    df = None 

Data loaded successfully.


In [5]:
if df is not None:
    USE_SEMANTIC_FEATURES = False
    
    print("\n" + "="*50)
    print(f"EXPERIMENT: Running with semantic features = {USE_SEMANTIC_FEATURES}")
    print("="*50)
    
    baseline_predictor = DeFiVolumePredictor(use_semantic_features=USE_SEMANTIC_FEATURES)
    X_train, X_test, y_train, y_test, features = baseline_predictor.prepare_data(df)
    print("Data preparation complete.")
    print(f"Training data shape: {X_train.shape}")
    print(f"Test data shape: {X_test.shape}")


EXPERIMENT: Running with semantic features = False
Data preparation complete.
Training data shape: (6145, 24)
Test data shape: (1537, 24)


In [6]:
if df is not None:
    print("\n" + "="*50)
    print("STEP 1: HYPERPARAMETER OPTIMIZATION WITH OPTUNA")
    print("="*50)
    
    baseline_predictor.optimize_model('rf', n_trials=30)

[I 2025-06-10 23:44:19,088] A new study created in memory with name: no-name-0899f84f-8c41-43fe-beda-2ad80c800f5c



STEP 1: HYPERPARAMETER OPTIMIZATION WITH OPTUNA

Optimizing RF...


[I 2025-06-10 23:44:27,368] Trial 0 finished with value: 0.12302623500587023 and parameters: {'n_estimators': 76, 'max_depth': 29, 'min_samples_split': 30, 'min_samples_leaf': 17, 'max_features': 'log2'}. Best is trial 0 with value: 0.12302623500587023.


Trial 0 for RF finished with value: 0.12303


[I 2025-06-10 23:44:42,442] Trial 1 finished with value: 0.12624432262633475 and parameters: {'n_estimators': 138, 'max_depth': 27, 'min_samples_split': 15, 'min_samples_leaf': 12, 'max_features': 'log2'}. Best is trial 0 with value: 0.12302623500587023.


...

Trial 27 for RF finished with value: 0.12431


[I 2025-06-10 23:52:30,112] Trial 28 finished with value: 0.12964533899238842 and parameters: {'n_estimators': 125, 'max_depth': 13, 'min_samples_split': 28, 'min_samples_leaf': 11, 'max_features': 'sqrt'}. Best is trial 26 with value: 0.12229929024650148.


Trial 28 for RF finished with value: 0.12965


[I 2025-06-10 23:52:43,070] Trial 29 finished with value: 0.12657661033994663 and parameters: {'n_estimators': 156, 'max_depth': 8, 'min_samples_split': 24, 'min_samples_leaf': 12, 'max_features': 'sqrt'}. Best is trial 26 with value: 0.12229929024650148.


Trial 29 for RF finished with value: 0.12658

Optimization for RF complete.
Best validation MAE: 0.1223
Best params: {'n_estimators': 82, 'max_depth': 8, 'min_samples_split': 27, 'min_samples_leaf': 10, 'max_features': 'sqrt'}


In [7]:
baseline_predictor.optimize_model('lstm', n_trials=15)

[I 2025-06-10 23:53:13,234] A new study created in memory with name: no-name-a4d7b265-71c8-42f7-9220-5b7fc2b28a04



Optimizing LSTM...


[I 2025-06-10 23:54:28,539] Trial 0 finished with value: 0.16053803265094757 and parameters: {'sequence_length': 24, 'learning_rate': 0.000991059225779319, 'dropout_rate': 0.46209930073199057, 'n_units_1': 256, 'n_units_2': 64}. Best is trial 0 with value: 0.16053803265094757.


Trial 0 for LSTM finished with value: 0.16054


[I 2025-06-10 23:55:38,740] Trial 1 finished with value: 0.3032906651496887 and parameters: {'sequence_length': 48, 'learning_rate': 0.00030501078468756653, 'dropout_rate': 0.29518345553307423, 'n_units_1': 256, 'n_units_2': 64}. Best is trial 0 with value: 0.16053803265094757.


...

[I 2025-06-11 00:10:47,371] Trial 13 finished with value: 0.11097129434347153 and parameters: {'sequence_length': 72, 'learning_rate': 0.009175744999229598, 'dropout_rate': 0.3737043035600437, 'n_units_1': 128, 'n_units_2': 64}. Best is trial 13 with value: 0.11097129434347153.


Trial 13 for LSTM finished with value: 0.11097


[I 2025-06-11 00:11:11,477] Trial 14 finished with value: 0.15547992289066315 and parameters: {'sequence_length': 72, 'learning_rate': 0.006482371449081739, 'dropout_rate': 0.37795690865687687, 'n_units_1': 64, 'n_units_2': 32}. Best is trial 13 with value: 0.11097129434347153.


Trial 14 for LSTM finished with value: 0.15548

Optimization for LSTM complete.
Best validation MAE: 0.1110
Best params: {'sequence_length': 72, 'learning_rate': 0.009175744999229598, 'dropout_rate': 0.3737043035600437, 'n_units_1': 128, 'n_units_2': 64}


{'sequence_length': 72,
 'learning_rate': 0.009175744999229598,
 'dropout_rate': 0.3737043035600437,
 'n_units_1': 128,
 'n_units_2': 64}

In [8]:
baseline_predictor.optimize_model('gru', n_trials=15)

[I 2025-06-11 00:11:30,448] A new study created in memory with name: no-name-d803e43f-bd12-4eb4-9013-7984d7a627a0



Optimizing GRU...


[I 2025-06-11 00:12:09,408] Trial 0 finished with value: 0.19026044011116028 and parameters: {'sequence_length': 72, 'learning_rate': 0.0008945622483090992, 'dropout_rate': 0.15017098481959812, 'n_units_1': 64, 'n_units_2': 32}. Best is trial 0 with value: 0.19026044011116028.


Trial 0 for GRU finished with value: 0.19026


[I 2025-06-11 00:13:07,164] Trial 1 finished with value: 0.1706087291240692 and parameters: {'sequence_length': 24, 'learning_rate': 0.00019736880006167935, 'dropout_rate': 0.14206733800029628, 'n_units_1': 256, 'n_units_2': 64}. Best is trial 1 with value: 0.1706087291240692.


...

[I 2025-06-11 00:22:39,439] Trial 13 finished with value: 0.19222764670848846 and parameters: {'sequence_length': 48, 'learning_rate': 0.0005098491608785031, 'dropout_rate': 0.3029826456778313, 'n_units_1': 64, 'n_units_2': 64}. Best is trial 5 with value: 0.11869289726018906.


Trial 13 for GRU finished with value: 0.19223


[I 2025-06-11 00:25:17,449] Trial 14 finished with value: 0.1494389921426773 and parameters: {'sequence_length': 72, 'learning_rate': 0.0015181480840605857, 'dropout_rate': 0.46788973204600537, 'n_units_1': 128, 'n_units_2': 32}. Best is trial 5 with value: 0.11869289726018906.


Trial 14 for GRU finished with value: 0.14944

Optimization for GRU complete.
Best validation MAE: 0.1187
Best params: {'sequence_length': 48, 'learning_rate': 0.0008783087279134427, 'dropout_rate': 0.49143379231925255, 'n_units_1': 64, 'n_units_2': 128}


{'sequence_length': 48,
 'learning_rate': 0.0008783087279134427,
 'dropout_rate': 0.49143379231925255,
 'n_units_1': 64,
 'n_units_2': 128}

In [9]:
if df is not None:
    USE_SEMANTIC_FEATURES = True
    
    print("\n" + "="*50)
    print(f"EXPERIMENT: Running with semantic features = {USE_SEMANTIC_FEATURES}")
    print("="*50)
    
    semantic_predictor = DeFiVolumePredictor(use_semantic_features=USE_SEMANTIC_FEATURES)
    X_train, X_test, y_train, y_test, features = semantic_predictor.prepare_data(df)
    print("Data preparation complete.")
    print(f"Training data shape: {X_train.shape}")
    print(f"Test data shape: {X_test.shape}")


EXPERIMENT: Running with semantic features = True
Data preparation complete.
Training data shape: (6145, 32)
Test data shape: (1537, 32)


In [10]:
if df is not None:
    print("\n" + "="*50)
    print("STEP 1: HYPERPARAMETER OPTIMIZATION WITH OPTUNA")
    print("="*50)
    
    semantic_predictor.optimize_model('rf', n_trials=30)

[I 2025-06-11 00:25:47,979] A new study created in memory with name: no-name-244337ca-0fc3-4cf7-a15d-40125e62f412



STEP 1: HYPERPARAMETER OPTIMIZATION WITH OPTUNA

Optimizing RF...


[I 2025-06-11 00:26:19,481] Trial 0 finished with value: 0.13214169602157502 and parameters: {'n_estimators': 254, 'max_depth': 20, 'min_samples_split': 25, 'min_samples_leaf': 5, 'max_features': 'log2'}. Best is trial 0 with value: 0.13214169602157502.


Trial 0 for RF finished with value: 0.13214


[I 2025-06-11 00:26:26,677] Trial 1 finished with value: 0.13034444215090482 and parameters: {'n_estimators': 58, 'max_depth': 27, 'min_samples_split': 18, 'min_samples_leaf': 16, 'max_features': 'log2'}. Best is trial 1 with value: 0.13034444215090482.


...

Trial 27 for RF finished with value: 0.12721


[I 2025-06-11 00:38:01,817] Trial 28 finished with value: 0.117976248376589 and parameters: {'n_estimators': 207, 'max_depth': 5, 'min_samples_split': 2, 'min_samples_leaf': 10, 'max_features': 'sqrt'}. Best is trial 28 with value: 0.117976248376589.


Trial 28 for RF finished with value: 0.11798


[I 2025-06-11 00:38:16,308] Trial 29 finished with value: 0.12356200917181641 and parameters: {'n_estimators': 168, 'max_depth': 7, 'min_samples_split': 19, 'min_samples_leaf': 11, 'max_features': 'sqrt'}. Best is trial 28 with value: 0.117976248376589.


Trial 29 for RF finished with value: 0.12356

Optimization for RF complete.
Best validation MAE: 0.1180
Best params: {'n_estimators': 207, 'max_depth': 5, 'min_samples_split': 2, 'min_samples_leaf': 10, 'max_features': 'sqrt'}


In [11]:
semantic_predictor.optimize_model('lstm', n_trials=15)

[I 2025-06-11 00:39:14,999] A new study created in memory with name: no-name-5f72062b-acd1-4fdf-9be3-694ed04f15e8



Optimizing LSTM...


[I 2025-06-11 00:39:46,921] Trial 0 finished with value: 0.1747954934835434 and parameters: {'sequence_length': 48, 'learning_rate': 0.0015150154130193266, 'dropout_rate': 0.4944920916377161, 'n_units_1': 64, 'n_units_2': 128}. Best is trial 0 with value: 0.1747954934835434.


Trial 0 for LSTM finished with value: 0.17480


[I 2025-06-11 00:41:09,027] Trial 1 finished with value: 0.18209978938102722 and parameters: {'sequence_length': 48, 'learning_rate': 0.0008036651196358127, 'dropout_rate': 0.2180795199569628, 'n_units_1': 256, 'n_units_2': 32}. Best is trial 0 with value: 0.1747954934835434.


...

[I 2025-06-11 00:56:55,861] Trial 13 finished with value: 0.13346756994724274 and parameters: {'sequence_length': 72, 'learning_rate': 0.0038873227378823814, 'dropout_rate': 0.23541996958621125, 'n_units_1': 256, 'n_units_2': 64}. Best is trial 11 with value: 0.1048518493771553.


Trial 13 for LSTM finished with value: 0.13347


[I 2025-06-11 00:58:24,783] Trial 14 finished with value: 0.13855579495429993 and parameters: {'sequence_length': 72, 'learning_rate': 0.0028310110938638217, 'dropout_rate': 0.2700979051681922, 'n_units_1': 128, 'n_units_2': 64}. Best is trial 11 with value: 0.1048518493771553.


Trial 14 for LSTM finished with value: 0.13856

Optimization for LSTM complete.
Best validation MAE: 0.1049
Best params: {'sequence_length': 72, 'learning_rate': 0.0052689088722282845, 'dropout_rate': 0.272542155247986, 'n_units_1': 128, 'n_units_2': 64}


{'sequence_length': 72,
 'learning_rate': 0.0052689088722282845,
 'dropout_rate': 0.272542155247986,
 'n_units_1': 128,
 'n_units_2': 64}

In [12]:
semantic_predictor.optimize_model('gru', n_trials=15)

[I 2025-06-11 00:58:34,782] A new study created in memory with name: no-name-37566d96-17ec-4bd7-88c6-9fb28b90a7a3



Optimizing GRU...


[I 2025-06-11 01:00:00,100] Trial 0 finished with value: 0.14531834423542023 and parameters: {'sequence_length': 48, 'learning_rate': 0.00260306528132184, 'dropout_rate': 0.32826480856119766, 'n_units_1': 64, 'n_units_2': 128}. Best is trial 0 with value: 0.14531834423542023.


Trial 0 for GRU finished with value: 0.14532


[I 2025-06-11 01:00:42,338] Trial 1 finished with value: 0.27003687620162964 and parameters: {'sequence_length': 72, 'learning_rate': 0.00018405130991291154, 'dropout_rate': 0.21386779247373325, 'n_units_1': 128, 'n_units_2': 32}. Best is trial 0 with value: 0.14531834423542023.


...

[I 2025-06-11 01:11:09,933] Trial 13 finished with value: 0.16074293851852417 and parameters: {'sequence_length': 48, 'learning_rate': 0.0036117062862611577, 'dropout_rate': 0.38051840970583206, 'n_units_1': 64, 'n_units_2': 128}. Best is trial 11 with value: 0.11475760489702225.


Trial 13 for GRU finished with value: 0.16074


[I 2025-06-11 01:11:51,385] Trial 14 finished with value: 0.18223527073860168 and parameters: {'sequence_length': 48, 'learning_rate': 0.001307727145664806, 'dropout_rate': 0.3256245111551736, 'n_units_1': 64, 'n_units_2': 128}. Best is trial 11 with value: 0.11475760489702225.


Trial 14 for GRU finished with value: 0.18224

Optimization for GRU complete.
Best validation MAE: 0.1148
Best params: {'sequence_length': 48, 'learning_rate': 0.009509241728516261, 'dropout_rate': 0.2832139771319348, 'n_units_1': 64, 'n_units_2': 128}


{'sequence_length': 48,
 'learning_rate': 0.009509241728516261,
 'dropout_rate': 0.2832139771319348,
 'n_units_1': 64,
 'n_units_2': 128}

In [13]:
if df is not None:
    print("\n" + "="*50)
    print("STEP 2: TRAINING FINAL BASELINE MODELS WITH OPTIMIZED PARAMETERS")
    print("="*50)
    baseline_predictor.train_model('rf', X_train, y_train, X_test, y_test)
    baseline_predictor.train_model('lstm', X_train, y_train, X_test, y_test)
    baseline_predictor.train_model('gru', X_train, y_train, X_test, y_test)


STEP 2: TRAINING FINAL BASELINE MODELS WITH OPTIMIZED PARAMETERS

=== Training RF with best parameters ===
Final Test Metrics -> MAE: 0.0360, RMSE: 0.0577, R²: 0.8563

=== Training LSTM with best parameters ===
Final Test Metrics -> MAE: 0.0725, RMSE: 0.0985, R²: 0.5742

=== Training GRU with best parameters ===
Final Test Metrics -> MAE: 0.0821, RMSE: 0.1036, R²: 0.5295


In [14]:
if df is not None:
    print("\n" + "="*50)
    print("STEP 2: TRAINING FINAL SEMANTIC MODELS WITH OPTIMIZED PARAMETERS")
    print("="*50)
    semantic_predictor.train_model('rf', X_train, y_train, X_test, y_test)
    semantic_predictor.train_model('lstm', X_train, y_train, X_test, y_test)
    semantic_predictor.train_model('gru', X_train, y_train, X_test, y_test)


STEP 2: TRAINING FINAL SEMANTIC MODELS WITH OPTIMIZED PARAMETERS

=== Training RF with best parameters ===
Final Test Metrics -> MAE: 0.0395, RMSE: 0.0591, R²: 0.8480

=== Training LSTM with best parameters ===
Final Test Metrics -> MAE: 0.0744, RMSE: 0.0911, R²: 0.6382

=== Training GRU with best parameters ===
Final Test Metrics -> MAE: 0.1571, RMSE: 0.1981, R²: -0.7745


In [15]:
if df is not None:
    print("\n" + "="*50)
    print("FINAL BASELINE RESULTS SUMMARY")
    print("="*50)
    results_df = pd.DataFrame(baseline_predictor.results).T
    print(results_df.sort_values(by='mae'))


FINAL BASELINE RESULTS SUMMARY
           mae      rmse        r2
rf    0.035995  0.057727  0.856289
lstm  0.072456  0.098458  0.574215
gru   0.082086  0.103564  0.529462


In [16]:
if df is not None:
    print("\n" + "="*50)
    print("FINAL SEMANTIC RESULTS SUMMARY")
    print("="*50)
    results_df = pd.DataFrame(semantic_predictor.results).T
    print(results_df.sort_values(by='mae'))


FINAL SEMANTIC RESULTS SUMMARY
           mae      rmse        r2
rf    0.039456  0.059101  0.847957
lstm  0.074422  0.091080  0.638226
gru   0.157058  0.198104 -0.774504


In [17]:
import joblib
import json
import os
from tensorflow.keras.models import load_model

def save_predictor_state(predictor, path_prefix):
    """Saves the predictor's state (models, scalers, parameters)."""
    
    # 1. Create the directory if it doesn't exist
    os.makedirs(path_prefix, exist_ok=True)
    print(f"Saving state to directory: {path_prefix}")

    # 2. Save the models
    for model_name, model in predictor.models.items():
        if model_name == 'rf':
            # Scikit-learn models are saved using joblib
            joblib.dump(model, os.path.join(path_prefix, f"{model_name}_model.joblib"))
        elif model_name in ['lstm', 'gru']:
            # Keras models are saved using their own method
            model.save(os.path.join(path_prefix, f"{model_name}_model.keras"))
    print("  - Models saved.")
            
    # 3. Save the scalers
    joblib.dump(predictor.scalers, os.path.join(path_prefix, "scalers.joblib"))
    print("  - Scalers saved.")

    # 4. Save the best parameters
    with open(os.path.join(path_prefix, "best_params.json"), 'w') as f:
        json.dump(predictor.best_params, f, indent=4)
    print("  - Best parameters saved.")

# --- Run the save process for both predictors ---
if 'baseline_predictor' in locals():
    save_predictor_state(baseline_predictor, "saved_models/baseline")

if 'semantic_predictor' in locals():
    save_predictor_state(semantic_predictor, "saved_models/semantic")

Saving state to directory: saved_models/baseline
  - Models saved.
  - Scalers saved.
  - Best parameters saved.
Saving state to directory: saved_models/semantic
  - Models saved.
  - Scalers saved.
  - Best parameters saved.
